<a href="https://colab.research.google.com/github/grfgrf/PySpark-Google-Colab/blob/main/03_Pyspark_SQL_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download/instalação dependências Spark/Pyspark
*primeira run aprox 60segundos

1.   apt-get openjdk-8-jdk-headless
2.   download spark-3.2.0-bin-hadoop3.2.tgz
3.   unzip spark-3.2.0-bin-hadoop3.2    
4.   pip install findspark (pyspark)
5.   set JAVA_HOME e SPARK_HOME paths
6.   cria variavel "spark" como sparkSession



In [1]:
%%bash
#***IFs apenas para o notebook não executar novamente em caso de run all cells.

#verifica se openjdk está instalado
if ! (dpkg -l | grep -qw openjdk-8-jdk-headless) then 
  apt-get install openjdk-8-jdk-headless -qq > /dev/null #| echo "openjdk-8-jdk-headless - instalado com sucesso" 
fi

In [2]:
%%bash
#download spark-hadoop
if ! [ -f "spark-3.2.0-bin-hadoop3.2.tgz" ]; then
  wget -q  https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz #| echo "spark-3.2.0-bin-hadoop3.2.tgz - baixado com sucesso"
fi

In [3]:
%%bash
#extrair spark-hadoop
if ! [ -d "spark-3.2.0-bin-hadoop3.2" ]; then
  tar xf spark-3.2.0-bin-hadoop3.2.tgz #| echo "spark-3.2.0-bin-hadoop3.2.tgz - descompactado com sucesso"
fi

In [4]:
#usa folder do spark como lib pyspark
try:
    findspark
except NameError:
    !pip install -q findspark
    import findspark
    findspark.init('spark-3.2.0-bin-hadoop3.2')
    #print("findspark - instalado com sucesso ")
#else:
    #print("já instalado - findspark")

In [5]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [6]:
#cria spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [81]:
from pyspark.sql.types import *#StrucktType StructField IntegerType StringType BooleanType
lista=[(1,'a',None),(2,'b',True),(3,'C',False),(2,'d',True),(2,'b',True)]

dfschema = StructType([
       StructField("Col_numero", IntegerType(), False),
       StructField("Col_letra", StringType(), False),
       StructField("Col_booleano",BooleanType(), True)
       ])
# Tipo do dado será wrapado em pyspark.sql.types.StructType
# caso não seja usado explicitamente

# Cria DataFrame
df = spark.createDataFrame(lista, dfschema)
df3 = df2 = df

# DataFrame(jdf, sql_ctx)
	

A distributed collection of data grouped into named columns.

In [73]:
df.collect()

[Row(Col_numero=1, Col_letra='a', Col_booleano=None),
 Row(Col_numero=2, Col_letra='b', Col_booleano=True),
 Row(Col_numero=3, Col_letra='C', Col_booleano=False),
 Row(Col_numero=2, Col_letra='d', Col_booleano=True),
 Row(Col_numero=2, Col_letra='b', Col_booleano=True)]

In [74]:
df.columns

['Col_numero', 'Col_letra', 'Col_booleano']

In [75]:
df.count()

5

In [76]:
dfdistinct = df.distinct()
dfdistinct.count()

4

In [77]:
df.dropna(how='any').orderBy('Col_numero',ascending=False).show()

+----------+---------+------------+
|Col_numero|Col_letra|Col_booleano|
+----------+---------+------------+
|         3|        C|       false|
|         2|        d|        true|
|         2|        b|        true|
|         2|        b|        true|
+----------+---------+------------+



In [78]:
df.fillna(False,'Col_booleano').show()

+----------+---------+------------+
|Col_numero|Col_letra|Col_booleano|
+----------+---------+------------+
|         1|        a|       false|
|         2|        b|        true|
|         3|        C|       false|
|         2|        d|        true|
|         2|        b|        true|
+----------+---------+------------+



In [79]:
from pyspark.sql.functions import column
df.select('Col_numero', 
          df['Col_numero'], 
          df.Col_numero,
          column('Col_numero')
         ).show()

+----------+----------+----------+----------+
|Col_numero|Col_numero|Col_numero|Col_numero|
+----------+----------+----------+----------+
|         1|         1|         1|         1|
|         2|         2|         2|         2|
|         3|         3|         3|         3|
|         2|         2|         2|         2|
|         2|         2|         2|         2|
+----------+----------+----------+----------+



In [80]:
df.replace(False,True).show()

+----------+---------+------------+
|Col_numero|Col_letra|Col_booleano|
+----------+---------+------------+
|         1|        a|        null|
|         2|        b|        true|
|         3|        C|        true|
|         2|        d|        true|
|         2|        b|        true|
+----------+---------+------------+



# Column(jc)
	

A column in a DataFrame.

In [16]:
#Column.astype() / cast()
lista = df.select(df['Col_numero'] ,#original int
                  df['Col_numero'].astype("string")
                 ).collect()[0]

print( type(lista[0]) , type(lista[1]) )

<class 'int'> <class 'str'>


In [17]:
# Column.desc_nulls_first()/ .asc_nulls_last()
df.select('Col_booleano').orderBy(df['Col_booleano'].asc_nulls_last()).collect()

[Row(Col_booleano=False),
 Row(Col_booleano=True),
 Row(Col_booleano=True),
 Row(Col_booleano=True),
 Row(Col_booleano=None)]

In [181]:
df.withColumn('Col_nova', df['Col_numero'] * df['Col_numero']).show()

+----------+---------+------------+--------+
|Col_numero|Col_letra|Col_booleano|Col_nova|
+----------+---------+------------+--------+
|         1|        a|        null|       1|
|         2|        b|        true|       4|
|         3|        C|       false|       9|
|         2|        d|        true|       4|
|         2|        b|        true|       4|
+----------+---------+------------+--------+



# GroupedData(jgd, df)
	

A set of methods for aggregations on a DataFrame, created by DataFrame.groupBy().

In [19]:
from pyspark.sql.functions import column
#  SELECT Col_boolean, COUNT(*) 
#    FROM df 
#GROUP BY Col_boolean 
#  HAVING COUNT(*) > 1

dfagg =df.groupBy('Col_booleano').count().where(column('count') > 1)
dfagg.show()


#from pyspark.sql.functions import count # count
#df.groupBy('Col_booleano').agg(count('Col_booleano').alias('count')).filter(column('count')>1).show()

+------------+-----+
|Col_booleano|count|
+------------+-----+
|        true|    3|
+------------+-----+



In [20]:
#   SELECT df1.col_numero, df2.count
#     FROM df1
#LEFT JOIN df2
#       ON df1.col_numero = df2.count  

df.join(dfagg ,on = df['Col_numero'] == dfagg['count'], how = 'left') \
  .select('Col_numero','count').show()

+----------+-----+
|Col_numero|count|
+----------+-----+
|         1| null|
|         2| null|
|         3|    3|
|         2| null|
|         2| null|
+----------+-----+



In [21]:
#  SELECT Col_numero, count(*)
#    FROM tabela
#GROUP BY CUBE(Col_numero)  
df.cube('Col_numero').count().show()

+----------+-----+
|Col_numero|count|
+----------+-----+
|         1|    1|
|         2|    3|
|         3|    1|
|      null|    5|
+----------+-----+



In [180]:
#  SELECT * 
#    FROM tabela1
#    JOIN tabela1
#   USING (Col_numero,Col_letra)

using_clause = ['Col_numero','Col_letra']

#Join ON  coluna que tem null ignora a row*
df.join(df ,using_clause).distinct().show()



+----------+---------+------------+------------+
|Col_numero|Col_letra|Col_booleano|Col_booleano|
+----------+---------+------------+------------+
|         1|        a|        null|        null|
|         2|        b|        true|        true|
|         2|        d|        true|        true|
|         3|        C|       false|       false|
+----------+---------+------------+------------+



# pyspark.sql.functions e outros

In [182]:
from pyspark.sql.functions import col
#renomeia colunas 1
df.select(*(col(x).alias('DF1_' + x) for x in df.columns)).show()

+--------------+-------------+----------------+
|DF1_Col_numero|DF1_Col_letra|DF1_Col_booleano|
+--------------+-------------+----------------+
|             1|            a|            null|
|             2|            b|            true|
|             3|            C|           false|
|             2|            d|            true|
|             2|            b|            true|
+--------------+-------------+----------------+



In [173]:
#renomeia colunas 2
df.selectExpr([nomecoluna + ' as DF1_' + nomecoluna for nomecoluna in df.columns]).show()

+--------------+-------------+----------------+
|DF1_Col_numero|DF1_Col_letra|DF1_Col_booleano|
+--------------+-------------+----------------+
|             1|            a|            null|
|             2|            b|            true|
|             3|            C|           false|
|             2|            d|            true|
|             2|            b|            true|
+--------------+-------------+----------------+

